Reference for image segmentation: 
1. [Code](https://github.com/nikhilroxtomar/UNet-Segmentation-in-Keras-TensorFlow/blob/master/unet-segmentation.ipynb)
2. Paper: [https://arxiv.org/abs/1505.04597](https://arxiv.org/abs/1505.04597)
3. Paper: [https://doi.org/10.1371/journal.pcbi.1005177](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005177)

Part 1: Image Segmentation by CNN

Imports:

In [2]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from skimage import img_as_float

import tensorflow as tf
from tensorflow import keras

import zipfile

In [ ]:
tf.__version__

Data generator

In [ ]:
class DataGen(keras.utils.Sequence):
  def __init__(self, ids, path, batch_size=4, image_size=576):
    self.ids = ids
    self.path = path
    self.batch_size = batch_size
    self.image_size = image_size
    self.on_epoch_end()

  def __load__(self, id_name):
    image_path_1 = os.path.join(self.path, id_name, "images/")
    #the path is corresponding the uploaded file names
    image_path_2 =[]
    for file in os.listdir(image_path_1):
      if file.endswith(".tif"): #file type corresponding to uploaded file type
        filename=os.path.join(image_path_1, file)
        image_path_2.append(filename)
    image_path =sorted(image_path_2)[0] #use [0] to take path of the only image 
    ##print (image_path)

    mask_path = os.path.join(self.path, id_name, "masks/")#
    all_masks=[]
    for file in os.listdir(mask_path):
      if file.endswith(".tif"):
        all_masks.append(file)

    #Reading images
    image = cv2.imread(image_path, 0) 
    #flag=1: depth=8, channel=3, but I would like to change it to grayscale image
    #if flag=1, then the image.shape=(577,577,3)
    #flg = 0, image.shape=(576, 576):grayscale image
    image = cv2.resize(image, (self.image_size, self.image_size))
    image = np.array(image) #make it as numpy array
    image=img_as_float(image) #change image type to float

    ## Reading Masks
    boundary_mask_path= mask_path+sorted(all_masks)[0]
    interior_mask_path=mask_path+sorted(all_masks)[1]
    junk_mask_path=mask_path+ sorted(all_masks)[2]
    #must be noticed that [0],[1],[2] corresponds to the name of the mask of 1 image;
    #file with "__bound": the first be extracted; __interior: 2nd; __junk: third

    ##print ("boundarymaskpath:"+boundary_mask_path)

    boundary_mask= cv2.imread(boundary_mask_path, 0)
    interior_mask= cv2.imread(interior_mask_path, 0)
    junk_mask=cv2.imread(junk_mask_path, 0)

    boundary_mask=np.array(boundary_mask)
    interior_mask=np.array(interior_mask)
    junk_mask=np.array(junk_mask)

    boundary=boundary_mask
    interior=interior_mask/2.0
    junk=junk_mask/4.0
    #chage the pixel value of the mask images to make different regions have different values:

    mask_0 = np.maximum(boundary, interior)
    mask = np.maximum(mask_0, junk)
    ##for use the boudanry only: 
    #mask = boundary_mask
    mask = cv2.resize(mask, (self.image_size, self.image_size))

    #Normalizing:
    image *= 255.0/image.max()
    image=image/255.0
    mask=mask/255.0  

    return image, mask

  def __getitem__(self, index):
    if(index+1)*self.batch_size > len(self.ids):
      self.batch_size = len(self.ids) - index*self.batch_size
    files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
    #print ("files_batch="+ str(files_batch))

    image = []
    mask = []

    for id_name in files_batch:
      _img, _msk = self.__load__(id_name)
      image.append(_img)
      mask.append(_msk)

    image = np.array(image)
    mask = np.array(mask)
    #a=image.shape
    #print ("shape of image"+str(a))

    return image, mask
  
  def on_epoch_end(self):
    pass
  
  def __len__(self):
    return int(np.ceil(len(self.ids)/float(self.batch_size)))
    #some bug are due to this code when training the model 


Unzip zipfile

In [ ]:
with zipfile.ZipFile('./content/images_training.zip', 'r') as zip_ref:
    zip_ref.extractall('./content')
#the path should be conrespond to the file name uploaded

In [ ]:
#the following check is used for check
os.listdir('./content/images_training/train/a6/masks')

Hyperparameters

In [ ]:
image_size = 576
# 576 is a good number for CNN
train_path ='./content/images_training/train'
#train_path corresponds to uploaded files
epochs=100
batch_size = 4

# Training ids
train_ids = next(os.walk(train_path))[1]
#return the file dir in train_path

## Validation Data size
val_data_size = 12 #need to change later

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [ ]:
print(train_ids)

In [ ]:
gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)
x, y = gen.__getitem__(0)
print(x.shape, y.shape)


In [ ]:
#show the train image with masks:
r = random.randint(0, len(x)-1)
# Not sure what are they doing here: random.randint
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)
ax = fig.add_subplot(1, 2, 1)
ax.imshow(x[r])
ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(y[r], (image_size, image_size)), cmap="gray")

Different Convolutional Blocks

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    #x: the input
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

UNet Model

In [ ]:
def UNet():
    f = [8, 16, 32, 64, 128]
    inputs = keras.layers.Input((576, 576, 1))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = UNet()
model.compile(optimizer="adam", loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), metrics=["accuracy"])
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, 
                    epochs=epochs)

Testing model

In [ ]:
#save the model for further independent use:
model.save('./content')

In [6]:
model = keras.models.load_model('./content')

In [ ]:
## Dataset for prediction
x, y = valid_gen.__getitem__(0)
result = model.predict(x)
result.shape
#result = result > 1

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 1)
ax.imshow(np.reshape(y[1]*255, (image_size, image_size)), cmap="gray")

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[1]*255, (image_size, image_size)), cmap="gray")

Part 2: Make Prediction: Image quantification

In [1]:
import skimage
from skimage import img_as_ubyte
from skimage import img_as_float
from skimage import morphology
from skimage.morphology import square
from skimage import measure
from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
#import image and normalize it:
Predicting_image_path='./content/vps26a-2 RGS1-YFP 0 min 6_ gluc 5.tif'
#notice: the image_path corresponds to certain image;
##for desinging user interface, image_path should be automatically as the imagepath of the image imported by the user.
##also could design for batch prediction

image_path=Predicting_image_path
image = cv2.imread(image_path, 0)
image = cv2.resize(image, (576, 576))
image=img_as_float(image)
image *= 255.0/image.max()
image=image/255.0
#related to whether the image normalized
image = np.array(image)


In [4]:
image.shape

(576, 576)

In [5]:
y = np.expand_dims(image, axis=0)
y.shape

(1, 576, 576)

In [7]:
result = model.predict(y)
result.shape

(1, 576, 576, 1)

In [ ]:
img=mpimg.imread(image_path)
imgplot = plt.imshow(img)
plt.show()

In [ ]:
fig = plt.figure()
fig.subplots_adjust(hspace=0.4, wspace=0.4)

ax = fig.add_subplot(1, 2, 2)
ax.imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")

Image quantificaton:

In [ ]:
result= np.squeeze(result, axis=0)

In [ ]:
result.shape

In [ ]:
result= np.squeeze(result, axis=-1)

In [ ]:
result.shape

In [ ]:
result

In [ ]:
plt.imshow(result, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
prediction_data=result

In [ ]:
#only show the boundary:
prediction_data[prediction_data >= 0.7] = 1
prediction_data[prediction_data < 0.7] = 0 

## Critical: must notice the output of CNN is pixel with continous numbers:
# higher accuracy, the closer it is to the pixel values designated in masks.
# thus, the number "0.6" made here is arbitary, depends on the accuracy of CNN, if Accuracy is high, it should be close to 1.0
### could influence the quantification result by thining or expending the boundary/membrane areas

## could also find ways to show other ROI predicted to make more options for the user.

In [ ]:
plt.imshow(prediction_data, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([]) 
plt.show()

In [ ]:
prediction_data

In [ ]:
prediction_data = prediction_data.astype('int')
prediction_data = morphology.remove_small_objects(prediction_data.astype(bool),64)
prediction_data = prediction_data.astype('float32') 

In [ ]:
prediction_data = skimage.morphology.closing(prediction_data, square(3))

In [ ]:
ROW= [0,1,2,3,4,5]
for I in ROW:
  prediction_data[I,:]=1
  prediction_data[:,I]=1
  prediction_data[-I,:]=1
  prediction_data[:,-I]=1

## add the row at the margin of image, because of bad annotation, the boundary in image margin would be miss classified.

In [ ]:
prediction_data[prediction_data == 1.0] = 0.5
prediction_data[prediction_data == 0.0] = 1.0
prediction_data[prediction_data == 0.5] = 0.0

#change the interior area to value 1

In [ ]:
s = generate_binary_structure(2,2)

In [ ]:
labeled_array, num_features = label(prediction_data, structure=s)

In [ ]:
num_features
#how many ROI/unconnected object it find

In [ ]:
plt.imshow(labeled_array)
#different colors conrespond to different labels (numbers)

In [ ]:
unique, counts = np.unique(labeled_array, return_counts=True)
dict(zip(unique, counts))
#label (number) has 26669 pixels 

In [ ]:
for i in range(num_features+1):
  if np.count_nonzero(labeled_array == i)>1000: #discard small ROI
    a = np.count_nonzero(labeled_array == i)
    print('pixel area =',np.count_nonzero(labeled_array == i))
    b = ndimage.sum(image, labeled_array, index=[i])
    print('pixel intensity =',ndimage.sum(image, labeled_array, index=[i]))
    arr_1 = (labeled_array == i).astype(int)
    a1=np.roll(arr_1, 8, axis=0)
    a2=np.roll(arr_1, -8, axis=0)
    a3=np.roll(arr_1, 8, axis=1)
    a4=np.roll(arr_1, -8, axis=1)
    a5=a1+a2+a3+a4
    a5[a5 > i-0.1] = 1
    c= ndimage.sum(image, a5, index=[1])-b
    print('pixel intensity in boundary =',c)
    ##Critical: the value "8" in np.rool meas to expand the region i by 8 pixels
    ## the value "8" corresponds to the length of the cell membranes.
    ##Critical: the prediction accuracy of CNN also would influence the parameter used in np.rool:
    ### also related to the parameter in "#only show the boundary:" lines

    slice_x, slice_y = ndimage.find_objects(labeled_array == i)[0]
    roi = labeled_array[slice_x, slice_y]
    plt.figure()
    plt.imshow(roi)
    print('------')
    #print(i)
    #print(slice_x)